In [ ]:
# !pip install estival

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

from estival.model import BayesianCompartmentalModel
from estival import priors as esp
from estival import targets as est

## Priors

Priors are simple!  They have a name, a distribution (based on their class), and some
parameters appropriate to the distribution

In [ ]:
p_uniform = esp.UniformPrior("p_uniform", [0.0, 1.0])
p_truncnormal = esp.TruncNormalPrior("p_truncnormal", mean=1.0, stdev=1.0, trunc_range=[0.0,np.inf])

In [ ]:
# Priors expose various internal information, but always provide 2 basic functions

# ppf - prior probability function; the value at a given quantile, for converting from uniform
#       to probability space

p_uniform.ppf(np.linspace(0.0,1.0,10))

In [ ]:
# Note that if we specify a prior with unbounded support, this will produce infinities as expected...
# care must be taken when sampling
p_truncnormal.ppf(np.linspace(0.0,1.0,10))

In [ ]:
# The bounds method takes a confidence interval, useful for clamping or setting sampling ranges

p_truncnormal.bounds(0.98)

In [ ]:
# logpdf - ...the logpdf

# Uniform... not very interesting
p_uniform.logpdf(0.5)

In [ ]:
# Plot the logpdf over a 95% ci
x = p_truncnormal.ppf(np.linspace(0.025,0.975,100))
pd.Series(p_truncnormal.logpdf(x), x).plot()

## Targets

In [ ]:
# Create some synthetic data

# First create a full date index, covering the complete domain of our 'model' (ie the domain of the data that
# will be passed to TargetEvaluators when we come to compute likelihood)
# In practice, when using BayesianCompartmenalModel, this will be obtained from the summer2 CompartmentalModel
full_date_index = pd.date_range(datetime(2001,1,1),datetime(2002,12,31),freq='d')

# Some random data
full_data = np.random.normal(size=len(full_date_index))

# Construct a pandas Series - estival is 'pandas native' and uses its types extensively
s = pd.Series(full_data, full_date_index)

# Now select a subset for use in our target - the kind of sparse gappy data that we might find in a real dataset
s_subset = s.iloc[::60]

s_subset.plot(style='.')

In [ ]:
# Construct the target
t = est.NormalTarget("test", s_subset, 1.0)

In [ ]:
# Import the Epoch class
# This is used anywhere we want to convert from numerical 'model time' to datetime

from summer2.utils import Epoch

In [ ]:
# Epochs take a single 'ref_date' (0.0 in model time), and an optional frequency argument; default is 1 day
epoch = Epoch(full_date_index[0])
epoch

In [ ]:
# Example usage
epoch.dti_to_index(s_subset.index)

In [ ]:
# When a BayesianCompartmenalModel is constructed, it will obtain the Epoch from the summer2 model,
# and then build evaluators for each of its targets
# This building step means all the subsetting logic (and any other possible pre-compuation)
# only needs to happen once, at the time of creation

teval = t.get_evaluator(full_date_index, epoch)

In [ ]:
# What if we didn't supply an epoch?

# t.get_evaluator(full_date_index, None)

In [ ]:
teval.index, teval.data

In [ ]:
# This is what gets called when a BayesianCompartmentalModel is asked to calculate a likelihood
# It takes the modelled data as input (over the full input domain), and handles the indexing etc internally
# Note the second parameters argument - it's empty for this target
teval.evaluate(full_data, {})

In [ ]:
# Construct a target with a dispersion parameter
t_dispersed = est.NormalTarget("test_dispersed", s_subset, esp.UniformPrior("dispersion_param", [0.01,1.0]))

In [ ]:
# Note that this new target now returns its dispersion parameter in get_priors;
# The BayesianCompartmentalModel obtains the full list of priors at the time of construction,
# so that these are always included in sampling
t_dispersed.get_priors()

In [ ]:
tdisp_eval = t_dispersed.get_evaluator(full_date_index, epoch)

In [ ]:
# When an external wrapper samples from a BayesianCompartmentalModel, it will use
# the full priors list, and sampled priors will be passed through to the evaluators
# as parameters 

tdisp_eval.evaluate(full_data + 0.1, {"dispersion_param": 0.1})

## BayesianCompartmentalModel

This is where it all comes together - the bridge between a summer2 model and the priors/targets we have described above

In [ ]:
from summer2.extras.test_models import sir

In [ ]:
m = sir()

In [ ]:
parameters = m.get_default_parameters()
parameters

In [ ]:
m.run(parameters)
m.get_derived_outputs_df().plot()

In [ ]:
notif_data_raw = m.get_derived_outputs_df()["notifications"].to_numpy()
notif_series = m.get_derived_outputs_df()["notifications"].iloc[:40:5]

In [ ]:
targets = [
    est.TruncatedNormalTarget("notifications", notif_series, [0.0, np.inf], esp.UniformPrior("notif_dispersion", [0.01,10.0]))
]

In [ ]:
priors = [
    esp.UniformPrior("contact_rate", [0.2,0.8])
]

In [ ]:
bcm = BayesianCompartmentalModel(m, parameters, priors, targets)

In [ ]:
# Priors
bcm.priors

In [ ]:
# Targets
bcm.targets

In [ ]:
# TargetEvaluators are created for us
bcm._evaluators["notifications"](notif_data_raw, {"notif_dispersion": 0.1})

In [ ]:
# Priors supply their logpdf...

bcm.priors["contact_rate"].logpdf(parameters["contact_rate"]) + bcm.priors["notif_dispersion"].logpdf(0.1)

In [ ]:
#...and the BayesianCompartmentalModel combines these...

bcm.run(parameters | {"notif_dispersion": 0.1}, include_extras=True).extras

In [ ]:
bcm.run(parameters | {"contact_rate": 0.5} | {"notif_dispersion": 0.1}, include_extras=True).extras

In [ ]:
bcm.run(parameters | {"contact_rate": 0.5} | {"notif_dispersion": 10.0}, include_extras=True).extras